In [1]:
import requests
import pandas as pd
import time

In [2]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': '9cdef0bc741e471385b100af843c28c5',
    'client_secret': 'a0820dd07a3e4d8facb51bb5ed5130a2',
})
# convert the response to JSON
auth_response_data = auth_response.json()
# save the access token
access_token = auth_response_data['access_token']
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [3]:
#get track audio features
def get_playlists(playlist_df,user_id):
    # pull all tracks from this album
    for user in user_id:
        user_id = user
        r = requests.get(BASE_URL + 'users/' + user_id + '/playlists/', headers=headers)
        d = r.json()
        time.sleep(1)
        
        i = 0
        for playlist in d['items']:
            if playlist['type'] == 'playlist':
                playlist_id = d['items'][i]['id']
                playlist_name = d['items'][i]['name']
                description = d['items'][i]['description']
                num_tracks = d['items'][i]['tracks']['total']
                owner = d['items'][i]['owner']['display_name']
                is_public = d['items'][i]['public']
                is_collaborative = d['items'][i]['collaborative']
        
                #save data in pd dataframe
                ##playlist_df = playlist_df.append({'playlist_id':playlist_id,'playlist_name': playlist_name,'description': description,
                ##    'num_tracks': num_tracks,'owner': owner,'is_public': is_public,'is_collaborative': is_collaborative},ignore_index=True)
                temp_df = pd.DataFrame({'playlist_id':playlist_id,'playlist_name': playlist_name,'description': description,
                    'num_tracks': num_tracks,'owner': owner,'is_public': is_public,'is_collaborative': is_collaborative},index=[i])
                playlist_df = pd.concat([playlist_df, temp_df], ignore_index=True)
            i += 1

    return playlist_df

In [4]:
#Main
BASE_URL = 'https://api.spotify.com/v1/'
user_id = ['1210694363']
#user_id = pd.read_csv(r'C:\Users\dsgra\Desktop\data science pj\user_id.csv')
#user_id = user_id.loc[:,'user_id']
playlist_df = pd.DataFrame(columns=["playlist_id","playlist_name","description","num_tracks","owner","is_public","is_collaborative"])
playlist_df = get_playlists(playlist_df, user_id)

playlist_df.to_csv(r'~/Documents/spotify_analysis/playlists.csv',encoding='utf-8-sig')
print('-----------------------------------')
print('Saved to .csv in project directory.')
print('-----------------------------------')

-----------------------------------
Saved to .csv in project directory.
-----------------------------------


In [5]:
#/v1/playlists/{playlist_id}/tracks
playlist_id = '1DkMMHspCJGfr3Fo0a2gjm'
f = requests.get(BASE_URL + 'playlists/' + playlist_id + '/tracks/', headers=headers)
a = f.json()

import json
with open('playlist_song_list.json', 'w', encoding='utf-8') as f:
    json.dump(a, f, ensure_ascii=False, indent=4)

In [6]:
# it is possible to get album and track info from the json of the playlist request, but not audio-features.

In [7]:
playlist_df

,playlist_id,playlist_name,description,num_tracks,owner,is_public,is_collaborative
0,5NeHy1JV3CpdvYuW2OtqH5,Old Gods of Asgard,,5,Derek Scott Grammer,True,False
1,2wqOdmAROa3ORY4vMzuAV6,Bluegrass Magic,,36,Derek Scott Grammer,True,False
2,1DkMMHspCJGfr3Fo0a2gjm,Derek's Easy Listening Playlist,Songs for when I am just not in the mood for t...,346,Derek Scott Grammer,True,False
3,3tSaOhLCDVn4v61sRprG6k,Video Game Music for Guitar,,8,Derek Scott Grammer,True,False
